In [1]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Linear Model

# Dataset

Attributes:

- CRIM: per capita crime rate by town.
- ZN: proportion of residential land zoned for lots over 25,000 sq.ft.
- INDUS: proportion of non-retail business acres per town.
- CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
- NOX: nitric oxides concentration (parts per 10 million).
- RM: average number of rooms per dwelling.
- AGE: proportion of owner-occupied units built prior to 1940.
- DIS: weighted distances to five Boston employment centers.
- RAD: index of accessibility to radial highways.
- TAX: full-value property-tax rate per \$10,000.
- PTRATIO: pupil-teacher ratio by town.
- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.
- LSTAT: \% lower status of the population.
- MEDV: Median value of owner-occupied homes in $1000's.

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv")
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [3]:
corr_mat = df.corr()
corr_mat

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
crim,1.000000,-0.200469,0.406583,-0.055892,0.420972,-0.219247,0.352734,-0.379670,0.625505,0.582764,0.289946,-0.385064,0.455621,-0.388305
zn,-0.200469,1.000000,-0.533828,-0.042697,-0.516604,0.311991,-0.569537,0.664408,-0.311948,-0.314563,-0.391679,0.175520,-0.412995,0.360445
indus,0.406583,-0.533828,1.000000,0.062938,0.763651,-0.391676,0.644779,-0.708027,0.595129,0.720760,0.383248,-0.356977,0.603800,-0.483725
chas,-0.055892,-0.042697,0.062938,1.000000,0.091203,0.091251,0.086518,-0.099176,-0.007368,-0.035587,-0.121515,0.048788,-0.053929,0.175260
nox,0.420972,-0.516604,0.763651,0.091203,1.000000,-0.302188,0.731470,-0.769230,0.611441,0.668023,0.188933,-0.380051,0.590879,-0.427321
rm,-0.219247,0.311991,-0.391676,0.091251,-0.302188,1.000000,-0.240265,0.205246,-0.209847,-0.292048,-0.355501,0.128069,-0.613808,0.695360
age,0.352734,-0.569537,0.644779,0.086518,0.731470,-0.240265,1.000000,-0.747881,0.456022,0.506456,0.261515,-0.273534,0.602339,-0.376955
dis,-0.379670,0.664408,-0.708027,-0.099176,-0.769230,0.205246,-0.747881,1.000000,-0.494588,-0.534432,-0.232471,0.291512,-0.496996,0.249929
rad,0.625505,-0.311948,0.595129,-0.007368,0.611441,-0.209847,0.456022,-0.494588,1.000000,0.910228,0.464741,-0.444413,0.488676,-0.381626
tax,0.582764,-0.314563,0.720760,-0.035587,0.668023,-0.292048,0.506456,-0.534432,0.910228,1.000000,0.460853,-0.441808,0.543993,-0.468536


In [4]:
correlations = sorted(corr_mat["medv"].items(),key=lambda x: -abs(x[1]))[1:]
correlations

[('lstat', -0.7376627261740145),
 ('rm', 0.6953599470715401),
 ('ptratio', -0.5077866855375623),
 ('indus', -0.48372516002837274),
 ('tax', -0.4685359335677667),
 ('nox', -0.42732077237328203),
 ('crim', -0.38830460858681154),
 ('rad', -0.38162623063977735),
 ('age', -0.3769545650045961),
 ('zn', 0.3604453424505433),
 ('b', 0.3334608196570662),
 ('dis', 0.249928734085904),
 ('chas', 0.17526017719029868)]

In [ ]:
attr_order = [attr for attr, _ in correlations]
sns.pairplot(df, x_vars=attr_order[:6], y_vars=['medv'])
sns.pairplot(df, x_vars=attr_order[6:], y_vars=['medv'])
plt.show()

# Train Test Split

In [ ]:
X = df[["lstat","rm","crim","age"]]
y = df["medv"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

In [ ]:
train_data = pd.concat([X_train, y_train], axis=1)

# Building The Model

In [ ]:
model = smf.ols(formula='medv ~ lstat + rm + age + crim', data=train_data).fit()

In [ ]:
model.summary()

# Validating

In [ ]:
preds = model.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
print("MSE",mse)
print("R^2",r2)

In [ ]:
sns.residplot(x=preds, y=y_test - preds, lowess=True, line_kws={'color': 'red'})
plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.title('Residual plot')


In [ ]:
sm.graphics.plot_regress_exog(model, 'lstat',)
sm.graphics.plot_regress_exog(model, 'crim',)
sm.graphics.plot_regress_exog(model, 'rm')
sm.graphics.plot_regress_exog(model, 'age')
plt.show()